Trying to load in the pdb file of ZIKV_NS2B3_ligands.pdb

In [ ]:
# I think I need these packages
#import pandas
#import numpy

# https://github.com/choderalab/asapdiscovery/blob/main/asapdiscovery-data/asapdiscovery/data/openeye.py
from asapdiscovery.data.openeye import oechem, load_openeye_pdb, save_openeye_sdf, openeye_perceive_residues
from asapdiscovery.modeling import modeling
from openeye import oechem # might want oespruce as well: https://docs.eyesopen.com/applications/spruce/spruce/spruce_overview.html
oechem.OEChemIsLicensed("python")

In [ ]:
# Load in the pdb file
zikv_file = "/Users/choderalab/asapdiscovery/temp_storage/ZIKV_NS2B3_ligands.pdb"
zikv_complex= load_openeye_pdb(zikv_file)
zikv_complex

Splitting function from: 

https://github.com/choderalab/asapdiscovery/blob/main/asapdiscovery-modeling/asapdiscovery/modeling/modeling.py#L567

In [ ]:
# documents: https://docs.eyesopen.com/toolkits/python/oechemtk/OEBioClasses/OESplitMolComplexOptions.html
# oechem.OESplitMolComplexOptions() <- Not going to use, but seems kind of fun

# Split the molecules
split_zikv = modeling.split_openeye_mol(zikv_complex)
split_zikv

Recombining the individual ligands and the protein

https://github.com/choderalab/asapdiscovery/blob/main/asapdiscovery-data/asapdiscovery/data/openeye.py#L25

In [ ]:
ligands = split_zikv['lig']
ligands.NumGroups()

In [ ]:
for i, atom in enumerate(ligands.GetAtoms()):
    print(i)

In [ ]:
# Try to see if this may be better
perceived_ligs = openeye_perceive_residues(ligands)
save_openeye_sdf(perceived_ligs, "perceived_ligs.sdf")

In [ ]:
# bonds is a list of tuples representing the bonds between atoms
bonds_list = []
for i, bond in enumerate(ligands.GetBonds()):
    bonds_list.append((bond.GetBgnIdx(), bond.GetEndIdx()))

In [ ]:
def cluster_atoms(bonds):
    atom_piles = {}
    pile_number = 1

    def dfs(atom, current_pile):
        if atom in atom_piles:
            return
        atom_piles[atom] = current_pile
        for bond in bonds:
            if atom == bond[0] and bond[1] not in atom_piles:
                dfs(bond[1], current_pile)
            elif atom == bond[1] and bond[0] not in atom_piles:
                dfs(bond[0], current_pile)

    for bond in bonds:
        start, end = bond
        if start not in atom_piles and end not in atom_piles:
            dfs(start, pile_number)
            pile_number += 1

    # Create a dictionary where keys are pile numbers and values are lists of atoms in each pile
    result_dict = {pile: [] for pile in set(atom_piles.values())}
    for atom, pile in atom_piles.items():
        result_dict[pile].append(atom)

    return result_dict

result = cluster_atoms(bonds_list)

print("Atom Piles:", result)


In [ ]:
# Want to do the following after
my_dict = {0: [0,1,2,3,4,5], 1: [6,7,8,9,10]}
new_ligs = {}
for lig_name, atom_idx in my_dict.items():
    new_lig = ligs.CreateCopy()
    for atom in new_lig.GetAtomIter():
        if not atom.GetIdx() in :
            new_lig.DeleteAtom(atom)
    new_ligs[lig_name] = new_lig

In [ ]:
# Neither of the things above work
# There are definitely more than 15 ligands (probably parsed out during reading or splitting)
# The way of deleting the atoms also probably wont work since it is OEMol objects are weird